In [22]:
from os.path import abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row

import collections
import io
import re
import tarfile
import os.path
import nltk
import smart_open
import gensim.utils

import pandas as pd
import pyodbc
import string

import gensim
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import pyLDAvis
#import pyLDAvis.gensim_models #Version 3.3 ?
import pyLDAvis.gensim

import pkgutil

import pkg_resources
from symspellpy import SymSpell, Verbosity

import sparknlp

import time
import pandas as pd
import pyodbc
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)
from nltk.stem.wordnet import WordNetLemmatizer
import string
import numpy as np
from gensim import corpora
pd.set_option('float_format', '{:f}'.format)
import spacy

import os

In [2]:
pkgutil.iter_modules(['pyLDAvis'])

pkgpath = os.path.dirname(pyLDAvis.__file__)
print([name for _, name, _ in pkgutil.iter_modules([pkgpath])])

['_display', '_prepare', '_server', 'gensim', 'graphlab', 'sklearn', 'urls', 'utils']


In [3]:

a = os.environ['SPARK_HOME']
a

'/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/lib/spark'

In [4]:

spark = SparkSession \
    .builder \
    .appName("test_spark_ch") \
    .enableHiveSupport() \
    .getOrCreate()

In [5]:
#os.listdir(os.getcwd())

In [6]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.yarn.appMasterEnv.MKL_NUM_THREADS', '1'),
 ('spark.driver.appUIAddress', 'http://10.64.73.82:20049'),
 ('spark.admin.acls.groups', 'edh_dev_yarn_admins,edh_dev_hdfs_data_admins'),
 ('spark.driver.blockManager.port', '25950'),
 ('spark.executor.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-7.1.7-1.cdh7.1.7.p0.15945976/lib/hadoop/lib/native'),
 ('spark.modify.acls.groups', 'edh_dev_yarn_admins,edh_dev_hdfs_data_admins'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter,'),
 ('spark.app.id', 'application_1640035010672_0010'),
 ('spark.sql.hive.convertMetastoreOrc', 'true'),
 ('spark.ui.view.acls.groups', 'edh_dev_yarn_admins,edh_dev_hdfs_data_admins'),
 ('spark.network.crypto.enabled', 'false'),
 ('spark.ui.proxyRedirectUri',
  'http://spark-xc3v591wvxb2lnmy.cdsw.hdp.amherst.com'),
 ('spark.executorEnv.MKL_NUM_THREADS', '1'),
 ('spark.sql.

In [7]:
#spark.catalog.listDatabases()

In [8]:
spark.sql("show databases").show(200)

+--------------------+
|        databaseName|
+--------------------+
|                 acs|
|              agency|
|        agency_model|
|        agency_stage|
|               altos|
|    amherst_rent_hpi|
|      attomjetstream|
|attomjetstream_stage|
|      bulkfinder_etl|
|bureau_economic_a...|
|bureau_economic_a...|
|bureau_labor_stat...|
|bureau_labor_stat...|
|                 cam|
|               carve|
|         carve_stage|
|              census|
|               cisco|
|         cisco_stage|
|           corelogic|
|     corelogic_stage|
|            dealpath|
|      dealpath_stage|
|             default|
|           demo_econ|
|                  dw|
|                embs|
|          embs_stage|
|           geography|
|     geography_stage|
|           gse_stage|
|  gud_etl_amhersthpi|
|     gud_etl_asgload|
|  gud_etl_clpropdata|
|  gud_etl_clprophist|
|gud_etl_evaluator_dw|
|         gud_etl_gud|
|     gud_etl_icbm_dw|
|         gud_etl_mls|
| gud_etl_msrpropdata|
|    gud_et

In [15]:
spark.sql('show tables in sandbox').show(200)

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| sandbox|raw_clearcapital_mls|      false|
| sandbox|    clearcapital_mls|      false|
| sandbox|irspublish_blockm...|      false|
| sandbox|irspublish_school...|      false|
| sandbox|irspublish_load_r...|      false|
| sandbox|modeltestbed_sfr_...|      false|
| sandbox|msrpropdata_asg_p...|      false|
| sandbox|mssql_thirdpartyd...|      false|
| sandbox|mssql_thirdpartyd...|      false|
| sandbox|mssql_amhersthpi_...|      false|
| sandbox|mssql_msrpropdata...|      false|
| sandbox|jf_dev_parcelid_t...|      false|
| sandbox|jm_propertybuyerh...|      false|
| sandbox|              taxrun|      false|
| sandbox|             taxrun2|      false|
| sandbox|       taxproperties|      false|
| sandbox|marketopportuniti...|      false|
| sandbox|marketopportuniti...|      false|
| sandbox|             taxrun3|      false|
| sandbox|    tax_analysis_232| 

In [17]:
#print(pyLDAvis.__version__)

df2 = spark.read.table('TV_AVMerr_RentSale_Subset') 

In [18]:
df2

DataFrame[listingid: bigint, asgpropid: double, qtr: bigint, tax_year: double, cbsa_div: string, cbsa: string, state: string, Transdate: bigint, Transprice: double, bedrooms: double, total_value: double, cj_living_area: double, basement: bigint, has_pool: bigint, parking: bigint, is_poor: boolean, is_good: bigint, HAS_VIEW: bigint, has_golf: boolean, has_water: boolean, has_woods: boolean, has_hill: boolean, fips_code: string, census_tract: string, zip: string, effective_year_built: double, story: double, ListingPrice: double, publicremarks: string, avmValue: double, avmerror: double, rentsale: string]

In [13]:
#[name for _, name, _ in pkgutil.iter_modules(['pyLDAvis'])]

In [9]:
# Data_frame with irspublish..TV_AVMerr_RentSale with

#stmt2 = """select * from irspublish..TV_AVMerr_RentSale with (Nolock) where rentsale='S'
#and cbsa in ('12060','19100','32820','16740','28140','45300','34980','26900',
#'36740','13820','38060','27260','36420','26420','31140','41180','41700','39580',
#'49180','35840','29460','33100','15980','37340','18140','24660','46060','19660',
#'29820','19740','20500','38940','33460','42660','42680','23580','40380','46220',
#'39460','36260','22180','17820','15500','41620','16980','39340','17900','27600',
#'14260','14500','24540','27180','12020','12680','31500','35380','34940','40660',
#'40900','46140','11500','20140','29300','23420','18620')"""


df_pandas = pd.read_feather('agent_comments.fea')

In [ ]:
df_pandas['closedprice']

In [ ]:
205148.850000	0.088227

In [ ]:
df = spark.createDataFrame(df_pandas) 
df.printSchema()
df.show()

In [12]:
df

DataFrame[listingid: bigint, asgpropid: double, qtr: bigint, tax_year: double, cbsa_div: string, cbsa: string, state: string, Transdate: bigint, Transprice: double, bedrooms: double, total_value: double, cj_living_area: double, basement: bigint, has_pool: bigint, parking: bigint, is_poor: boolean, is_good: bigint, HAS_VIEW: bigint, has_golf: boolean, has_water: boolean, has_woods: boolean, has_hill: boolean, fips_code: string, census_tract: string, zip: string, effective_year_built: double, story: double, ListingPrice: double, publicremarks: string, avmValue: double, avmerror: double, rentsale: string]

In [13]:
df.createOrReplaceTempView("TV_AVMerr_RentSale_Subset")

In [ ]:
from pyspark import SparkContext
import pyspark.sql 
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark)
sqlContext.sql("create table sandbox.TV_AVMerr_RentSale_Subset as select * from TV_AVMerr_RentSale_Subset");

In [ ]:
sqlContext.sql

In [51]:
# Spark data_frame with nlp_test_with_text


# THIS TABLE SEEMS TO BE CORRUPTED
df = spark.read.table('sandbox.nlp_test_with_text') 

In [52]:
df.columns

['id',
 'listingid',
 'mls',
 'mlslistingid',
 'unittype',
 'unit',
 'city',
 'state',
 'zip',
 'subdivision',
 'fips',
 'censustractgeoid',
 'schooldistrict',
 'propertytype',
 'propertysubtype',
 'propertydescription',
 'lotsizeacres',
 'lotsizesqft',
 'zoning',
 'restrictions',
 'easements',
 'watersource',
 'septicsewer',
 'gatedcommunity',
 'hoa',
 'hoaname',
 'hoamanagementco',
 'hoamanagementcophone',
 'occupanttype',
 'ownershiptype',
 'ownertype',
 'ownername',
 'ownerphone',
 'yearbuilt',
 'yearupdated',
 'numberofunits',
 'livingareasqft',
 'livingareasqftsource',
 'buildingstyle',
 'stories',
 'beds',
 'fullbaths',
 'halfbaths',
 'basement',
 'finishedbasementpct',
 'garagetype',
 'garagestyle',
 'garagespaces',
 'rooftype',
 'exteriormaterial',
 'foundation',
 'pool',
 'condition',
 'propertytaxappraisal',
 'propertytax',
 'propertytaxyear',
 'hoadues',
 'hoaduesfrequency',
 'hoaduesdescription',
 'rentsale',
 'entrydate',
 'listingdate',
 'listingstatus',
 'listingstatusd

In [53]:
df.printSchema()


root
 |-- id: long (nullable = true)
 |-- listingid: long (nullable = true)
 |-- mls: string (nullable = true)
 |-- mlslistingid: string (nullable = true)
 |-- unittype: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: long (nullable = true)
 |-- subdivision: string (nullable = true)
 |-- fips: long (nullable = true)
 |-- censustractgeoid: string (nullable = true)
 |-- schooldistrict: string (nullable = true)
 |-- propertytype: string (nullable = true)
 |-- propertysubtype: string (nullable = true)
 |-- propertydescription: string (nullable = true)
 |-- lotsizeacres: double (nullable = true)
 |-- lotsizesqft: double (nullable = true)
 |-- zoning: string (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- easements: string (nullable = true)
 |-- watersource: string (nullable = true)
 |-- septicsewer: string (nullable = true)
 |-- gatedcommunity: string (nullable = true)
 |-- hoa

In [14]:

print((df.count(), len(df.columns)))

KeyboardInterrupt: 

In [25]:
df.select('rentsale').distinct().collect()

[Row(rentsale='Community Pool'),
 Row(rentsale='"24-Hour Guard'),
 Row(rentsale='151.0'),
 Row(rentsale='102.0'),
 Row(rentsale='602.0'),
 Row(rentsale='247.0'),
 Row(rentsale='886.0'),
 Row(rentsale='"ParkPlayground'),
 Row(rentsale='2834.0'),
 Row(rentsale='143.92'),
 Row(rentsale='262.7'),
 Row(rentsale='7184.06'),
 Row(rentsale='1.0'),
 Row(rentsale='360.0'),
 Row(rentsale='302.0'),
 Row(rentsale='6102.0'),
 Row(rentsale='47.5'),
 Row(rentsale='414.37'),
 Row(rentsale='717.56'),
 Row(rentsale='353.16'),
 Row(rentsale='4848.0'),
 Row(rentsale='2624.37'),
 Row(rentsale='884.0'),
 Row(rentsale='"Boat Dock'),
 Row(rentsale='1588.37'),
 Row(rentsale='353.0'),
 Row(rentsale='Other-See Remarks'),
 Row(rentsale='5046.91'),
 Row(rentsale='115.76'),
 Row(rentsale='5062.0'),
 Row(rentsale='1839.0'),
 Row(rentsale='"Irrigation'),
 Row(rentsale='685.0'),
 Row(rentsale='Other-See Remarks"'),
 Row(rentsale='4092.0'),
 Row(rentsale='3963.0'),
 Row(rentsale='385.25'),
 Row(rentsale='1542.0'),
 Row(

In [ ]:

df.select("rentsale").show(30)

In [30]:
# Filter table by State GA and by Sale

df_sale_ST = df.filter((df.rentsale == 'S') & (df.state == 'GA')).collect()
df_rent_ST = df.filter((df.rentsale == 'R') & (df.state == 'GA')).collect()


In [31]:

print((df_sale.count(), len(df_sale.columns)))

(498615, 117)
(328994, 117)


In [23]:
len(df_sale)

TypeError: object of type 'DataFrame' has no len()

In [ ]:
.collect()

In [20]:
df[['censustractgeoid','propertydescription','rentsale','closedprice']].show(vertical=True)

-RECORD 0-----------------------------------
 censustractgeoid    | null                 
 propertydescription | Single Family Det... 
 rentsale            | Swimming             
 closedprice         | null                 
-RECORD 1-----------------------------------
 censustractgeoid    | null                 
 propertydescription | Single Family Det... 
 rentsale            | S                    
 closedprice         | 685000.0             
-RECORD 2-----------------------------------
 censustractgeoid    | null                 
 propertydescription | Single Family Det... 
 rentsale            | S                    
 closedprice         | 307000.0             
-RECORD 3-----------------------------------
 censustractgeoid    | null                 
 propertydescription | Single Family Res... 
 rentsale            | S                    
 closedprice         | 307000.0             
-RECORD 4-----------------------------------
 censustractgeoid    | null                 
 propertyd

In [ ]:
df[['censustractgeoid','propertydescription','rentsale']].show(vertical=True)